In [1]:
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline

from pprint import pprint
from zen_garden.postprocess.results import Results

from pathlib import Path
from os import chdir

In [3]:
# set path
path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/notebooks')
chdir(path)

In [4]:
# cost optimal solution
res_basic = Results("../data/outputs/03_example_CCTS")

In [5]:
# show the deafult analysis settings
pprint(res_basic.results[None]["analysis"])

/Users/davidbertschinger/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers/pydev/pydevd_plugins/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').declare_namespace(__name__)

/Users/davidbertschinger/Applications/PyCharm Professional Edition.app/Contents/plugins/python/helpers/pydev/pydevd_plugins/extensions/__init__.py:2: DeprecationWarning: Deprecated call to `pkg_resources.declare_namespace('pydevd_plugins.extensions')`.
Implementing implicit namespace packages (as specified in PEP 420) is preferred to `pkg_resources.declare_namespace`. See https://setuptools.pypa.io/en/latest/references/keywords.html#keyword-namespace-packages
  __import__('pkg_resources').decla

{'dataset': '/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/03_example_CCTS',
 'earliest_year_of_data': 1900,
 'folder_output': '/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/outputs',
 'header_data_inputs': {'set_capacity_types': 'capacity_type',
                        'set_carriers': 'carrier',
                        'set_conversion_technologies': 'technology',
                        'set_dependent_carriers': 'carrier',
                        'set_edges': 'edge',
                        'set_elements': 'element',
                        'set_failure_states': 'failure_states',
                        'set_input_carriers': 'carrier',
                        'set_lca_impact_categories': 'impact_category',
                        'set_location': 'location',
                        'set_nodes': 'node',
                        'set_output_carriers': 'carrier',
                        'set_retrofitting_technologies': 'technology',
                        'set_stor

In [6]:
res_basic.get_df("flow_conversion_input")

technology         carrier           node              time_operation
capture_amine      flue_gas          swiss_storage_CH  0                 0.0
                                                       1                 0.0
                                                       2                 0.0
                                                       3                 0.0
                                                       4                 0.0
                                                                        ... 
permanent_storage  co2_liquid_16bar  waste_2_CH        0                 0.0
                                                       1                 0.0
                                                       2                 0.0
                                                       3                 0.0
                                                       4                 0.0
Name: flow_conversion_input, Length: 255, dtype: float64

In [7]:
# get flow transport and save as csv
res_basic.get_df('flow_conversion_input', to_csv="flow_conversion_input")

technology         carrier           node              time_operation
capture_amine      flue_gas          swiss_storage_CH  0                 0.0
                                                       1                 0.0
                                                       2                 0.0
                                                       3                 0.0
                                                       4                 0.0
                                                                        ... 
permanent_storage  co2_liquid_16bar  waste_2_CH        0                 0.0
                                                       1                 0.0
                                                       2                 0.0
                                                       3                 0.0
                                                       4                 0.0
Name: flow_conversion_input, Length: 255, dtype: float64

In [8]:
# get flow transport csv
df = pd.read_csv('flow_conversion_input.csv')

In [9]:
df

,technology,carrier,node,time_operation,flow_conversion_input
0,capture_amine,flue_gas,swiss_storage_CH,0,0.0
1,capture_amine,flue_gas,swiss_storage_CH,1,0.0
2,capture_amine,flue_gas,swiss_storage_CH,2,0.0
3,capture_amine,flue_gas,swiss_storage_CH,3,0.0
4,capture_amine,flue_gas,swiss_storage_CH,4,0.0
...,...,...,...,...,...
250,permanent_storage,co2_liquid_16bar,waste_2_CH,0,0.0
251,permanent_storage,co2_liquid_16bar,waste_2_CH,1,0.0
252,permanent_storage,co2_liquid_16bar,waste_2_CH,2,0.0
253,permanent_storage,co2_liquid_16bar,waste_2_CH,3,0.0


In [10]:
# find unique technologies
technology = df['technology'].unique()
technology

array(['capture_amine', 'compression_16_110bar', 'condition_110bar',
       'condition_16bar', 'decompression_110_16bar',
       'decompression_110bar_ambient', 'emergency_storage', 'emitter_wte',
       'heat_pump', 'permanent_storage'], dtype=object)

In [11]:
# create nominal flow transport files

for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/03_example_CCTS_N-1_01_T')
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/nominal_flow_conversion_input.csv')
    
    #Clean up table
    x = x.drop(columns='technology')
    x = x[x['flow_conversion_input'] != 0]
    x = x.rename(columns={'time_operation': 'year'})
    x['year'] = x['year'] + 2025
    
    # Pivot the DataFrame
    x = x.pivot(index=['node', 'year'], columns='carrier', values='flow_conversion_input').reset_index()
    x.columns.name = None  # Remove the 'carrier' label on columns
    
    #clean up table, differentiate between node and edge
    if path.parent.name == 'set_transport_technologies':
        x = x.rename(columns={'location': 'edge'})

    else:
        x = x.rename(columns={'location': 'node'})

    #Safe
    x.to_csv(filepath, index=False)

In [25]:
for i in range(len(technology)):
     
    #Get table of one technology 
    tec = technology[i]
    x = df[df['technology'] == tec]
    
    #Set path
    parent_path = Path('/Users/davidbertschinger/Documents/GitHub/ZEN-garden/data/03_example_CCTS_N-1_addition_0')
    path = [file for file in Path(parent_path).rglob(tec)]
    path = path[0]
    filepath = Path(str(path)+'/nominal_flow_conversion_input.csv')
    
    #Clean up table
    x = x.drop(columns='technology')
    x = x[x['flow_conversion_input'] != 0]
    x = x.rename(columns={'time_operation': 'year'})
    x['year'] = x['year'] + 2025
    x

In [29]:
x
# Pivot the DataFrame
x_pivoted = x.pivot(index=['node', 'year'], columns='carrier', values='flow_conversion_input').reset_index()
x_pivoted.columns.name = None  # Remove the 'carrier' label on columns
print(x_pivoted)


               node  year  co2_liquid_16bar  electricity
0  swiss_storage_CH  2026         17.567023     0.594307
1  swiss_storage_CH  2027         17.567023     0.594307
2  swiss_storage_CH  2028         30.095759     1.018164
3  swiss_storage_CH  2029         30.095759     1.018164
